# Обучение моделей

## Импорт библиотек

In [1]:
import os, sys

PROJECT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(PROJECT_DIR)
sys.path.append(PROJECT_DIR)

/workspaces/street_tree


In [2]:
import pandas as pd 
from src.config import PATH_RAW, PATH_PROCESSED, URL, NAME_DATASET, PATH_MODELS, output_path
from src.dowload_datafraime import download_file
from src.visualization import plot_corr_matrix
from src.preprocessing import (df_fillna, split_problems, convert_to_bool, encode_and_save_categorical, 
                               load_and_encode_categorical, split_and_save)


2025-03-23 06:21:13,956 - src.utils - INFO - Logger is ready
2025-03-23 06:21:13,971 - src.utils - INFO - This is a test log message.
2025-03-23 06:21:13,973 - src.utils - INFO - PROJECT_DIR: /workspaces/street_tree
2025-03-23 06:21:13,977 - src.utils - INFO - PATH_LOGS: /workspaces/street_tree/logs
2025-03-23 06:21:13,981 - src.utils - INFO - PATH_DATA: /workspaces/street_tree/data
2025-03-23 06:21:13,983 - src.utils - INFO - PATH_RAW: /workspaces/street_tree/data/raw
2025-03-23 06:21:13,986 - src.utils - INFO - PATH_PROCESSED: /workspaces/street_tree/data/processed
2025-03-23 06:21:13,988 - src.utils - INFO - PATH_MODELS: /workspaces/street_tree/models
2025-03-23 06:21:13,993 - src.utils - INFO - PATH_REPORTS: /workspaces/street_tree/reports
2025-03-23 06:21:13,996 - src.utils - INFO - PATH_DOCS: /workspaces/street_tree/docs
2025-03-23 06:21:14,001 - src.utils - INFO - PATH_SRC: /workspaces/street_tree/src
2025-03-23 06:21:14,005 - src.utils - INFO - output_path: /workspaces/street_t

## Предобработка

In [3]:
df = pd.read_csv(f'{PATH_RAW}/{NAME_DATASET}.csv')
df.head(3)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl
0,180683,348711,2015-08-27T00:00:00.000,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03T00:00:00.000,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05T00:00:00.000,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34.0,449.0,3338310.0,3.028870e+09


In [4]:
df.columns = [col.lower().replace(' ', '_') for col in df.columns]
df_drop_columns = ['status', 'state', 'tree_id', 'created_at', 'stump_diam', 'address', 'spc_common',
                   'borocode', 'x_sp', 'y_sp', 'council_district', 'census_tract', 'nta_name',
                   'bin', 'zip_city', 'community_board', 'bbl']
df = df.drop(columns=df_drop_columns)
df.dropna(subset=['health'], inplace=True)
df.shape


(652172, 28)

In [5]:
df = df_fillna(df)
df.shape

/workspaces/street_tree/src/preprocessing.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['spc_latin'].fillna('No observation',inplace=True)
/workspaces/street_tree/src/preprocessing.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].metho

(652172, 28)

In [6]:
df = split_problems(df)
df.shape

2025-03-23 06:21:22,352 - src.utils - INFO - {'trunklights', 'branchother', 'rootother', 'stones', 'wiresrope', 'branchlights', 'trunkother', 'metalgrates', 'noproblem', 'sneakers'} - уникальные проблемы


(652172, 38)

In [7]:
df = convert_to_bool(df)
df.shape

2025-03-23 06:21:26,974 - src.utils - INFO - Значения преобразованы в булевые


(652172, 38)

In [8]:
drop_columns = ['trunklights', 'metalgrates', 'branchother', 'sneakers', 'wiresrope', 
    'branchlights', 'noproblem', 'rootother', 'stones', 'trunkother']
df = df.drop(columns=drop_columns)
df.shape

(652172, 28)

In [9]:
categorical_columns = df.select_dtypes(include='object').columns.tolist()
categorical_columns

['health', 'spc_latin', 'steward', 'guards', 'user_type', 'borough', 'nta']

In [10]:
df, label_encoders = encode_and_save_categorical(df, categorical_columns, PATH_MODELS)
df.shape

(652172, 28)

In [11]:
# Вывод значений, закодированных энкодером для столбца 'health', вместе с соответствующими им числовыми кодами
health_classes = label_encoders['health'].classes_
health_codes = label_encoders['health'].transform(health_classes)

# Создание DataFrame для удобного отображения
health_mapping = pd.DataFrame({'Health': health_classes, 'Code': health_codes})
print(health_mapping)

  Health  Code
0   Fair     0
1   Good     1
2   Poor     2


## Разделим на выборки

In [12]:

X_train, X_val, X_test, y_train, y_val, y_test = split_and_save(df, PATH_PROCESSED)

2025-03-23 06:21:36,334 - src.utils - INFO - Data successfully saved to: /workspaces/street_tree/data/processed


## Обучим модели

In [13]:
y_train.unique()

array([1, 2, 0])

In [14]:
from src.modeling import DeepLearningModel

model = DeepLearningModel(
    input_size=X_train.shape[1],
    hidden_layers=[512, 256, 128, 64], 
    learning_rate=0.0001,              
    model_dir=PATH_MODELS,
    y_train=y_train
)

model.train(X_train, y_train, X_val, y_val, num_epochs=5)

2025-03-23 06:21:40.646485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742710900.765099   44239 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742710900.814826   44239 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742710901.513774   44239 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742710901.513807   44239 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742710901.513811   44239 computation_placer.cc:177] computation placer alr

ImportError: cannot import name 'FocalLoss' from 'focal_loss' (/workspaces/street_tree/.venv/lib/python3.12/site-packages/focal_loss/__init__.py)